In [1]:
### Imports
import pandas as pd
import numpy as np
import pickle
import json
import argparse

# Imports DL
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from sklearn.metrics import *
import torch.utils.data as data
from torch.utils.data import Dataset
from hypnettorch.hnets import HyperNetInterface
from hypnettorch.hnets import HMLP
import copy
import time

from helpers_task_detector import *

/home/nerea/anaconda3/envs/sinthlab/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
template =  Causal_Simple_RNN(num_features=128, 
                    hidden_units= 300, 
                    num_layers = 1, 
                    out_dims = 2,
                    dropout = 0.2).to(device)
RNN_model =  Causal_Simple_RNN(num_features=128, 
                    hidden_units= 500, 
                    num_layers = 13, 
                    out_dims = 2,
                    dropout = 0.2).to(device)    

#### Defining the template, main and hnet models and initializing them

# We can use the task detector as a template for the main model
param_shapes = [p.shape for p in list(template.parameters())]

num_conditions = 60 # we want more possible conditions than what we can reach
size_task_embedding = 24 # changed to test on 22/07 it was 8 # seemed to work well 

hnet = HMLP(param_shapes, uncond_in_size=0,
            cond_in_size=size_task_embedding, #usually we had size_task_embedding
            layers= [50,50], # trying different values on 22/07, before that it was [13]
            num_cond_embs=num_conditions).to(device)

for param in hnet.parameters():
    param.requires_grad = True
for param in RNN_model.parameters():
    param.requires_grad = True

Created MLP Hypernet.
Hypernetwork with 6614942 weights and 129602 outputs (compression ratio: 51.04).
The network consists of 6613502 unconditional weights (6613502 internally maintained) and 1440 conditional weights (1440 internally maintained).


In [15]:
def count_hypernetwork_parameters(hnet):
    return sum(p.numel() for p in hnet.parameters() if p.requires_grad)

# Example usage:
num_params_1 = count_hypernetwork_parameters(hnet)
print(f"Number of learnable parameters in the hypernetwork: {num_params_1}")

Number of learnable parameters in the hypernetwork: 6614942


In [16]:
# Example usage:
num_params_2 = count_hypernetwork_parameters(RNN_model)
print(f"Number of learnable parameters in the hypernetwork: {num_params_2}")

Number of learnable parameters in the hypernetwork: 6328002


In [17]:
num_params_1/num_params_2

1.0453444863007313